In [5]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [ ]:
#!pip3 install --upgrade pip
# !pip install google-cloud-bigquery
# !pip install pandas-gbq -U
# !pip install db-dtypes
# !pip install packaging --upgrade


# Set the environment variable for Google Cloud credentials
# Place the path in which the .json file is located.

# Example (if .json is located in the same directory with the notebook)
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "at-arch-416714-6f9900ec7.json"

# -- YOUR CODE GOES BELOW THIS LINE
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/armela/Desktop/neat-calculus-453319-f5-08fadb59b569.json" # Edit path
# -- YOUR CODE GOES ABOVE THIS LINE

In [ ]:
# SQL Query to create table and insert data

create_and_insert_query = """
-- Create table if it doesn't exist
CREATE TABLE IF NOT EXISTS `neat-calculus-453319-f5.reporting_db.rep_revenue_per_period` (
  reporting_period STRING,
  reporting_date DATE,
  total_revenue NUMERIC
);

###Insert aggregated revenue per period from 2015 to current

INSERT INTO `neat-calculus-453319-f5.reporting_db.rep_revenue_per_period`
(reporting_period, reporting_date, total_revenue)

WITH payments AS (
  SELECT *
  FROM `neat-calculus-453319-f5.staging_db.stg_payment`
),

rentals AS (
  SELECT *
  FROM `neat-calculus-453319-f5.staging_db.stg_rental`
),

inventory AS (
  SELECT *
  FROM `neat-calculus-453319-f5.staging_db.stg_inventory`
),

films AS (
  SELECT *
  FROM `neat-calculus-453319-f5.staging_db.stg_film`
),

valid_payments AS (
  SELECT
    p.payment_amount,
    p.payment_date
  FROM payments p
  JOIN rentals r ON p.rental_id = r.rental_id
  JOIN inventory i ON r.inventory_id = i.inventory_id
  JOIN films f ON i.film_id = f.film_id
  WHERE f.film_title != 'GOODFELLAS SALUTE'
),

revenue_by_period AS (
  SELECT 'Day' AS reporting_period, DATE_TRUNC(payment_date, DAY) AS reporting_date, SUM(payment_amount) AS total_revenue
  FROM valid_payments
  GROUP BY 1, 2

  UNION ALL

  SELECT 'Month', DATE_TRUNC(payment_date, MONTH), SUM(payment_amount)
  FROM valid_payments
  GROUP BY 1, 2

  UNION ALL

  SELECT 'Year', DATE_TRUNC(payment_date, YEAR), SUM(payment_amount)
  FROM valid_payments
  GROUP BY 1, 2
),

final AS (
  SELECT
    rpt.reporting_period,
    rpt.reporting_date,
    COALESCE(rbp.total_revenue, 0) AS total_revenue
  FROM `neat-calculus-453319-f5.reporting_db.reporting_periods_table` rpt
  LEFT JOIN revenue_by_period rbp
    ON rpt.reporting_period = rbp.reporting_period
   AND rpt.reporting_date = rbp.reporting_date
  WHERE rpt.reporting_date >= '2015-01-01'
)


SELECT * 
FROM neat-calculus-453319-f5.reporting_db.rep_revenue_per_period
"""

# Run the query
query_job = client.query(create_and_insert_query)

# Wait for the query to finish
query_job.result()

# Print success message

print("Table 'rep_revenue_per_period' created and data inserted successfully!")